In [1]:
import pandas as pd
import pickle
from anytree import Node, RenderTree
import json
from pprint import pprint

In [2]:
with open("category_tree.pkl", 'rb') as f:
    data = pickle.load(f)

In [3]:
#for pre, fill, node in RenderTree(data[-1]):
#     print("%s%s" % (pre, node.name))

In [4]:
products = pd.read_csv("products.csv", index_col=0)
products_categories = pd.read_csv("products_categories.csv", index_col=0)

### Создаем из 420_searches.json таблицу searches


- **query** запрос<br>
- **external_id** id первого продукта, на кoторый кликнул покупатель

In [5]:
with open("420_searches.json") as ff:
    data = json.load(ff)
    pprint(data[:1])

[{'amount': 1,
  'products': {'89072600012': '1',
               '89072600013': '1',
               '89072600014': '1',
               '89072600015': '1',
               '89072600018': '1',
               '89072600019': '1',
               '89072600020': '1',
               '89072600021': '1',
               '89072600022': '1',
               '89072600023': '1',
               '89072600024': '1'},
  'query': '0 70'}]


In [6]:
query = []
product_external_id = []
for i in range(len(data[:1500])):
    query.append(list(data[i]['products'].keys())[0])
    product_external_id.append(data[i]['query'])
    i+=1
    #print(data[i]['query'], list(data[i]['products'].keys())[0])

In [7]:
searches = pd.DataFrame({'query': product_external_id, 'external_id': query})
searches.head(10)

,query,external_id
0,0 70,89072600018
1,2 molecules,26500100001
2,acqua di gio profumo,84141400001
3,acqua di parma blu mediterraneo bergamotto di ...,26422400004
4,acqua di parma capri,26421100011
5,anne semonin express radiance ice cube,88860200004
6,antipodes,30150400008
7,apple box filorga,19760308921
8,armani basi,80130600013
9,armani easy,24877600005


In [8]:
searches['external_id'] = pd.to_numeric(searches['external_id'])

### Создаем функцию, которая соединяет products и products_categories



In [9]:
def merge_product_external_id_to_categories(products, products_categories):
    df = pd.merge(products[['id', 'external_id']], products_categories, left_on='id', right_on='product_id')
    return df

In [10]:
products_categories_id = merge_product_external_id_to_categories(products, products_categories)
products_categories_id.head()

,id,external_id,site_id,product_id,category_id
0,2573904266,24944900083,420,2573904266,873827
1,2573904322,24944900078,420,2573904322,873827
2,2573904375,7005200014,420,2573904375,873827
3,2573904490,24944900057,420,2573904490,873827
4,2573905005,17059600013,420,2573905005,873828


### Добавляем каждому товару в таблицу searches его категорию

In [11]:
searches = pd.merge(searches, products_categories_id, on='external_id')
searches.head(10)

,query,external_id,id,site_id,product_id,category_id
0,0 70,89072600018,2579840720,420,2579840720,873895
1,0 70,89072600018,2579840720,420,2579840720,3672618
2,cle de p,89072600018,2579840720,420,2579840720,873895
3,cle de p,89072600018,2579840720,420,2579840720,3672618
4,2 molecules,26500100001,2573929893,420,2573929893,2987634
5,2 molecules,26500100001,2573929893,420,2573929893,3115485
6,2 molecules,26500100001,2573929893,420,2573929893,2987633
7,2 molecules,26500100001,2573929893,420,2573929893,3672618
8,acqua di gio profumo,84141400001,2573926076,420,2573926076,3115490
9,acqua di gio profumo,84141400001,2573926076,420,2573926076,3672618


В итоге у нас получилась таблица, где есть категория товара по каждому запросу. Далее необходимо добавить чужую категорию из дерева решений для каждой строки.